<a href="https://colab.research.google.com/github/P2Enjoy/ia-web3.fr/blob/main/SpeedWatch_GPU.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title Install all requirements (run once)
import locale
locale.getpreferredencoding = lambda: "UTF-8"

!pip install --upgrade transformers torch youtube_transcript_api accelerate bitsandbytes tensorflow fastpunct nnsplit onnxruntime-gpu tiktoken


import nltk, os
nltk.download('punkt')
os.environ['CUDA_LAUNCH_BLOCKING'] = "1"

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 24.6 MB/s eta 0:00:00


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [ ]:
#@title Load the model into memory
from transformers import pipeline

model_id = "facebook/bart-large-cnn" #@param {type:"string"}
summarizeryt = pipeline('summarization', model= model_id, device=0)

In [ ]:
#@title Load the model to restore transcripts punctuations
from transformers import pipeline

model_id = "SJ-Ray/Re-Punctuate" #@param {type:"string"}
repunctuaute = pipeline('summarization', model= model_id, device=0)

All model checkpoint layers were used when initializing TFT5ForConditionalGeneration.

All the layers of TFT5ForConditionalGeneration were initialized from the model checkpoint at SJ-Ray/Re-Punctuate.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFT5ForConditionalGeneration for predictions without further training.


In [ ]:
#@title Process the youtube video

#@markdown Put the link of the video to summarize:
video_link = "https://www.youtube.com/watch?v=jQL0ZeHtXFc" #@param {type:"string"}

def YTVideoToText(video_link):
  # installing & importing libraries
  from youtube_transcript_api import YouTubeTranscriptApi
  from fastpunct import FastPunct

  # fetching video transcript
  video_id = video_link.split("=")[1]
  transcript = YouTubeTranscriptApi.get_transcript(video_id)

  # The default language is 'english'
  fastpunct = FastPunct()
  # iterating throughout and adding all text together (recovering ponctuations)
  flat_transcript = ""
  for tu in transcript:
      flat_transcript += ' ' + tu['text']

  return flat_transcript

video_full_text = YTVideoToText(video_link)

In [ ]:
from nnsplit import NNSplit
splitter = NNSplit.load("en")

# returns `Split` objects
video_sentences = splitter.split([video_full_text])[0]

for sentence in video_sentences:
  import tiktoken

  encoding = tiktoken.encoding_for_model("gpt2")
  num_tokens = len(encoding.encode(f"{sentence}"))
  print(f"{sentence}; {num_tokens}")

 hello Community welcome to the most watched part of how to instruct fine tune your large language model after the amazing success of part 1 and part 2 now part three ; 32
so here we are now ; 6
we have here a look at the prompt Stanford alpaca prompt ; 14
so we are taking you exactly this prompt remember ; 10
this is step one like I showed you ; 9
now we do it one by one ; 8
so we say yeah let's execute ; 8
this have a look at it so we say okay ; 11
you're asked to come up with 20 diverse task ; 11
instruction now create Step 1 20 in structure and as I told you ; 15
there are some requirements try not to repeat the verb ; 11
the language should be diverse ; 6
the type of instruction should be diverse ; 8
the language chat GPT or GPT language suit model should be able to complete the instruction it will be in English ; 24
the instructions should be one to two sentences ; 9
long you should generate an appropriate input to the instruction ; 11
not all reinstructions require input that's 

In [ ]:
#@title Default title text
summarize_token_width = 500 #@param {type:"slider", min:250, max:2048, step:250}

def do_sum(sentences):
  # summarize text
  summarized_text = ''
  for sentence in sentences:
      out = summarizeryt(sentence, min_length=30, do_sample=False)
      out = out[0]
      out = out['summary_text']
      summarized_text.append(' ').append(out)

  # returning summary
  return summarized_text;

In [ ]:
#@title Show the transcript of the video
video_full_text

" hello Community welcome to the most watched part of how to instruct fine tune your large language model after the amazing success of part 1 and part 2 now part three so here we are now we have here a look at the prompt Stanford alpaca prompt so we are taking you exactly this prompt remember this is step one like I showed you now we do it one by one so we say yeah let's execute this have a look at it so we say okay you're asked to come up with 20 diverse task instruction now create Step 1 20 in structure and as I told you there are some requirements try not to repeat the verb the language should be diverse the type of instruction should be diverse the language chat GPT or GPT language suit model should be able to complete the instruction it will be in English the instructions should be one to two sentences long you should generate an appropriate input to the instruction not all reinstructions require input that's exactly as I showed you in my last video what is the highest peak in the

In [ ]:
#@title Show the summarized text of the video
sum_text